In [7]:
from src.preprocessor.utils.dataset_level import read_pickle, prepare_training_dataset, read_json
from itertools import islice

# corpus = read_pickle('/home/thiendc/projects/legal_retrieval/data/processed/corpus.pkl')
queries = read_pickle('/home/thiendc/projects/legal_retrieval/data/processed/queries.pkl')
# relevant_docs = read_pickle('/home/thiendc/projects/legal_retrieval/data/processed/relevant_docs.pkl')

In [11]:
queries = list(queries.values())

In [12]:
for i in queries:
    if "Nếu".lower() in i.lower():
        print(i)
        break

Sản phẩm phần mềm có được hưởng ưu đãi về thời gian miễn thuế, giảm thuế hay không? Nếu được thì trong vòng bao nhiêu năm?


In [24]:
from src.generator.gen_label.settings import DEFAULT_SYSTEM_PROMPT
DEFAULT_SYSTEM_PROMPT

' Bạn là người trợ lý hữu ích, trung thực và tài giỏi về lĩnh vực pháp luật dân sự Việt Nam. \nBạn hãy giúp tôi phân loại câu hỏi mà người dùng đưa vào để nhân diện chúng thuộc về loại câu hỏi nào trong 7 loại câu hỏi về pháp luật dưới đây:\n-   Câu hỏi pháp lý\n-   Câu hỏi về giải thích luật \n-   Câu hỏi về áp dụng luật\n-   Câu hỏi về quyền và nghĩa vụ \n-   Câu hỏi về tiền lệ pháp lý \n-   Câu hỏi về chứng cứ\n-   Câu hỏi tình huống" \n\nNếu câu hỏi người dùng đưa vào khó để phân loại, hãy gán nhãn là Unknown.\nHãy đưa câu trả lời dưới dạng json như ví dụ sau với mỗi user prompt\n{\n    "label": Câu hỏi tình huống,\n    "question": Câu hỏi được đưa vào để kiểm tra \n}\n\n'

In [27]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "Qwen/Qwen2.5-1.5B-Instruct"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype="float32",
#     device_map="auto"
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Sản phẩm phần mềm có được hưởng ưu đãi về thời gian miễn thuế, giảm thuế hay không? Nếu được thì trong vòng bao nhiêu năm?"
messages = [
    {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens= 128
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [26]:
text

'<|im_start|>system\n Bạn là người trợ lý hữu ích, trung thực và tài giỏi về lĩnh vực pháp luật dân sự Việt Nam. \nBạn hãy giúp tôi phân loại câu hỏi mà người dùng đưa vào để nhân diện chúng thuộc về loại câu hỏi nào trong 7 loại câu hỏi về pháp luật dưới đây:\n-   Câu hỏi pháp lý\n-   Câu hỏi về giải thích luật \n-   Câu hỏi về áp dụng luật\n-   Câu hỏi về quyền và nghĩa vụ \n-   Câu hỏi về tiền lệ pháp lý \n-   Câu hỏi về chứng cứ\n-   Câu hỏi tình huống" \n\nNếu câu hỏi người dùng đưa vào khó để phân loại, hãy gán nhãn là Unknown.\nHãy đưa câu trả lời dưới dạng json như ví dụ sau với mỗi user prompt\n{\n    "label": Câu hỏi tình huống,\n    "question": Câu hỏi được đưa vào để kiểm tra \n}\n\n<|im_end|>\n<|im_start|>user\nSản phẩm phần mềm có được hưởng ưu đãi về thời gian miễn thuế, giảm thuế hay không? Nếu được thì trong vòng bao nhiêu năm?<|im_end|>\n<|im_start|>assistant\n'

In [28]:
response

'{\n    "label": "Câu hỏi về quyền và nghĩa vụ",\n    "question": "Sản phẩm phần mềm có được hưởng ưu đãi về thời gian miễn thuế, giảm thuế hay không? Nếu được thì trong vòng bao nhiêu năm?"\n}'